In [1]:
import pandas as pd
import numpy as np
import glob

import pandas as pd
import numpy as np
import glob

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# convert 'Account' column to upper case
acct_desc = pd.read_csv(r'C:\Users\Family PC\Desktop\pythonfiles\callreport\AcctDesc.txt')
acct_desc['Account'] = acct_desc['Account'].str.upper()


In [3]:
# get data file names
path = r'C:\Users\Family PC\Desktop\pythonfiles\callreport' 
filenames = glob.glob(path + "/*.xlsx")

dfs = [] # an empty list to store the dataframes

for filename in filenames:
    df = pd.read_excel(filename) # read excel file
    dfs.append(df) # append to list

# concatenate all dataframes in the list and sort by CU_NUMBER and CYCLE_DATE
big_df = pd.concat(dfs, ignore_index=True)
big_df['CYCLE_DATE'] = pd.to_datetime(big_df['CYCLE_DATE']) #format='%m/%d/%Y',errors='coerce'
big_df = big_df.sort_values(['CU_NUMBER', 'CYCLE_DATE']).reset_index()

big_df['Prior_Year'] = big_df['CYCLE_DATE'].apply(lambda x: x.year - 1)

big_df = big_df.dropna(subset=['CU_NAME'])


In [4]:
# filter big_df for rows where CYCLE_DATE is a 12/31 date
dec_31_df = big_df[big_df['CYCLE_DATE'].dt.month == 12] 

# select only the CU_NUMBER, CYCLE_DATE, and ACCT_010 columns
dec_31_df = dec_31_df[['CU_NUMBER', 'CYCLE_DATE', 'ACCT_010']] 

# add a new column for the year of the CYCLE_DATE
dec_31_df['Year'] = dec_31_df['CYCLE_DATE'].dt.year 


In [5]:
# create a DataFrame with only the necessary columns from dec_31_df
dec_31_assets = dec_31_df[['CU_NUMBER', 'Year', 'ACCT_010']].copy()

# add a Cycle_Year column to dec_31_assets
dec_31_assets['Cycle_Year'] = dec_31_assets['Year'] + 1

# rename ACCT_010 to ACCT_010_y in dec_31_assets
dec_31_assets = dec_31_assets.rename(columns={'ACCT_010': 'PYE_Assets'})

# convert Year to int
dec_31_assets['Year'] = dec_31_assets['Year'].astype(int)

# add a CYCLE_DATE column to dec_31_assets with December 31 of the Year
dec_31_assets['CYCLE_DATE'] = pd.to_datetime(dec_31_assets['Year'].astype(str) + '-12-31')



In [6]:
# create Cycle_Year column in big_df based on the year of CYCLE_DATE
big_df['Cycle_Year'] = big_df['CYCLE_DATE'].dt.year

# merge big_df with dec_31_assets on CU_NUMBER and Cycle_Year
big_df = big_df.merge(dec_31_assets[['CU_NUMBER', 'Cycle_Year', 'PYE_Assets']], on=['CU_NUMBER', 'Cycle_Year'], how='left')

# if PYE_Assets is not already in big_df, add it
if 'PYE_Assets' not in big_df.columns:
    big_df['PYE_Assets'] = np.nan

# calculate Average_Assets
big_df['Average_Assets'] = (big_df['ACCT_010'] + big_df['PYE_Assets']) / 2



In [7]:
# make all column names uppercase
big_df.columns = map(str.upper, big_df.columns)

# modify column names
new_col_names = {}
for col in big_df.columns:
    if col != 'ACCOUNT': # skip the 'Account' column
        # get the corresponding account name from the AcctDesc.txt file
        try:
            acct_name = acct_desc[acct_desc['Account'] == col]['AcctName'].values[0]
        except IndexError:
            # handle index error if the account does not exist in AcctDesc.txt
            print(f"Warning: Account {col} does not exist in AcctDesc.txt")
            continue
        # blend the account name with the existing column name
        new_col_name = f"{acct_name} ({col})"
        new_col_names[col] = new_col_name

big_df.rename(columns=new_col_names, inplace=True)

big_df = big_df.drop('INDEX', axis=1)


# save the modified dataframe to csv
big_df.to_csv(r'C:\Users\Family PC\Desktop\pythonfiles\callreport\data\master_call_report_data.csv',index=False)


In [8]:
big_df.head(10)

,CU_NUMBER,CYCLE_DATE,JOIN_NUMBER,CU_NAME,CITY,STATE,TOTAL ASSETS (ACCT_010),Total Amount of Shares (ACCT_013),Total number of Loans and Leases (ACCT_025A),Total amount of Loans and Leases (ACCT_025B),Amount of Loans Granted Year-to-Date (ACCT_031B),Total Amount of Delinquent Loans & Leases (Two or more months) (ACCT_041B),Number of current members (not number of accounts) (ACCT_083),Provision for Loan & Lease Losses (ACCT_300),Interest on Borrowed Money (ACCT_340),Dividends on Shares (Includes dividends earned during current period) (ACCT_380),Non-Member Deposits (ACCT_457),Interest Rate of New Vehicle Loans (ACCT_523),Interest Rate of Used Vehicle Loans (ACCT_524),Total Amount of All Loans Charged Off Year-to-Date (ACCT_550),Net Income (unless the amount is already included in Undivided Earnings) (ACCT_602),Amount of Regular Shares (ACCT_657),Total Non-Interest Expense (Sum of items 18-27) (ACCT_671),Total Borrowings (ACCT_860C),Number of Used Vehicle Loans (ACCT_968),(Less) Interest Refunded (ACCT_119),Fee Income (ACCT_131),Employee Compensation and Benefits (ACCT_210),Travel and Conference Expense (ACCT_230),Office Occupancy Expense (ACCT_250),Office Operations Expense (ACCT_260),Educational and Promotional Expense (ACCT_270),Loan Servicing Expense (ACCT_280),Professional and Outside Services (ACCT_290),Member Insurance (ACCT_310),Operating Fees (Examination and/or supervision fees) (ACCT_320),Miscellaneous Operating Expenses (ACCT_360),Amount of Used Vehicle Loans (ACCT_370),Interest on Deposits (Total interest expense for deposit accounts) (SCU Only) (ACCT_381),Amount of New Vehicle Loans (ACCT_385),Amount of Unsecured Credit Card Loans (ACCT_396),Amount of All Other Unsecured Loans/Lines of Credit (ACCT_397),Number of Accounts for Share Drafts (ACCT_452),IRA/KEOGH Accounts (ACCT_453),Number of Accounts for Regular Shares (ACCT_454),All other shares (ACCT_455),Money Market Shares (ACCT_458),Number of Accounts for Total Shares and Deposits (Sum of items 15-16) (ACCT_460),Total Interest Income (ACCT_115),Total Non-Interest Income (ACCT_117),Total Interest Expense (Sum of items 6-8) (ACCT_350),Net Income (Loss) (line 11 plus line 17 less line 28) (ACCT_661A),Average of Daily Assets over the calendar quarter (ACCT_010A),Cash on Hand (Coin and Currency) (ACCT_730A),Total Cash on Deposit (Amounts Deposited in Financial Institutions) (ACCT_730B),Total Net Worth (ACCT_997),Net Worth Ratio (ACCT_998),Number of Outstanding Indirect Loans (ACCT_617A),Total Amount of Outstanding Indirect Loans (ACCT_618A),Total Shares (ACCT_966),Total Uninsured Member Shares and Deposits > $250K (A+A1+B+C+D+E) (ACCT_065A4),Total Uninsured Nonmember Shares and Deposits > $250K (G+H+I) (ACCT_067A2),Total Uninsured Shares and Deposits > $250K (F+J) (ACCT_068A),Total Insured Shares and Deposits > $250K (item 17 less item K) (ACCT_069A),Amount of Participation Loans Purchased Year-to-Date (ACCT_690),Amount of Participation Loans Sold Year-to-Date (ACCT_691),Cash on Deposit in Corporate Credit Unions (ACCT_730B1),Cash on Deposit in Other Financial Institutions (ACCT_730B2),Net Worth Classification if credit union is not new (Based upon Call Report data only. See instructions) (ACCT_700),Equity Acquired in Merger (ACCT_658A),Total Outstanding Amount of Participation Loans Purchased By Type (ACCT_691L),Total Value of Investments in CUSOs (ACCT_851),Total Amount Loaned to CUSOs (ACCT_852),Total Aggregate Cash Outlay in CUSOs (ACCT_853),Amount of Payday Alternative Loans (PAL loans) (ACCT_397A),Amount of Total Loans/Lines of Credit Secured by Junior Lien 1-4 Family Residential Properties (ACCT_386A),Amount of All Other Real Estate Loans/Lines of Credit (ACCT_386B),Cash on Deposit in the Federal Reserve Bank (ACCT_AS0003),Coin and Currency (ACCT_AS0004),Cash Items in Process of Collection (ACCT_AS0005),(CECL) indicator for early adoption of CECL (ACCT_AS0010),ACCT_AS0007,ACCT_AS0008,ACCT_AS0009,ACCT_AS0013,ACCT_AS0016,ACCT_AS0017,ACCT_AS0036,ACCT_DL0002,ACCT_D

In [12]:
acct_desc.head()

,Account,AcctName,AcctDesc,TableName,Annualize,CBSAccount,FPRAccount,STATAcctMain,STATAcctState,STATAcctGraph,STATAcctTable8
0,ACCT_001,Amount - Total Other Loans (Purchased or to no...,Include the dollar amount of outstanding loans...,FS220A,False,True,True,True,True,False,False
1,ACCT_002,Amount of Leases Receivable,"Leases Receivable. Include the rate, number, a...",FS220A,False,False,False,False,False,False,False
2,ACCT_003,Loans Held for Sale,Loans Held for Sale. Report the amount of outs...,FS220C,False,False,False,False,False,False,False
3,ACCT_007,Land and Building,"Land and building, less depreciation on buildi...",FS220,False,True,True,True,True,False,True
4,ACCT_008,Other Fixed Assets,All other fixed assets such as furniture and f...,FS220,False,True,True,True,True,False,True
